In [7]:
import numpy as np
from ClassicalApproach import *

In [8]:
measurements = []

with open("189W63-ZOU.csv", "r") as file:
    # skip first line
    file.readline()
    for index, line in enumerate(file.readlines()):
        parts = line.split(",")
        start          = parts[0]    # = start
        eind           = parts[1]    # = eind
        afstand        = parts[4]    # = afstand
        hoogteverschil = parts[5]    # = hoogteverschil
        std_a          = parts[7]    # = std_a
        std_b          = parts[8]    # = std_b
        std_c          = parts[9]    # = std_c
        measurement = Measurement(index=index,start=start,eind=eind,afstand=afstand,hoogteverschil=hoogteverschil,std_a=std_a,std_b=std_b,std_c=std_c)
        measurements.append(measurement)
        
    project = Measurements(measurements)
    
    var = project.create_var()
    print(var)

AttributeError: 'Measurements' object has no attribute 'lengt'

In [9]:
import numpy as np
np.set_printoptions(suppress=True)

x_i = np.array([
    [-6.230],
    [ 3.092],
    [ 3.139],
    [ 3.144],
    [-2.122],
    [-4.163],
    [ 3.410],
    [-3.406]
])

g_ij = np.array([
    [4, 0, 0, 0, 0, 0, 0, 0],
    [0, 4, 0, 0, 0, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 0, 0],
    [0, 0, 0, 4, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0],
    [0, 0, 0, 0, 0, 4, 0, 0],
    [0, 0, 0, 0, 0, 0, 4, 0],
    [0, 0, 0, 0, 0, 0, 0, 4],
])



u_i = np.array([
    [1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 1, 0, 0, 0, 0, 0],
])

u_o = np.array([
    [0],
    [0],
    [0],
    [3.142],
])

print(u_i.shape)  # 4x8
print(x_i.shape)  # 8x1
print(u_o.shape)  # 4x1
print(g_ij.shape) # 8x8 matrix

y_e = u_i.dot(x_i) - u_o

print(g_ij.shape) # 4x1 vector

# g_ij moet worden genormaliseerd

g_et = (u_i @ g_ij @ np.transpose(u_i)) / 4

print(g_et)

invers_g_et = np.linalg.inv(g_et)

print(invers_g_et)

e_i = (g_ij / 4) @ np.transpose(u_i) @ invers_g_et @ (-1 * y_e)

print(e_i)

x_i = x_i + e_i

print(x_i)

(4, 8)
(8, 1)
(4, 1)
(8, 8)
(8, 8)
[[3. 1. 0. 1.]
 [1. 4. 0. 1.]
 [0. 0. 2. 0.]
 [1. 1. 0. 1.]]
[[ 0.5         0.          0.         -0.5       ]
 [ 0.          0.33333333  0.         -0.33333333]
 [ 0.          0.          0.5         0.        ]
 [-0.5        -0.33333333  0.          1.83333333]]
[[-0.002     ]
 [-0.002     ]
 [ 0.003     ]
 [-0.00033333]
 [-0.00033333]
 [-0.00033333]
 [-0.002     ]
 [-0.002     ]]
[[-6.232     ]
 [ 3.09      ]
 [ 3.142     ]
 [ 3.14366667]
 [-2.12233333]
 [-4.16333333]
 [ 3.408     ]
 [-3.408     ]]


In [10]:
import numpy as np

def adjacency_to_constraint_matrix(A):
    nodes = A.shape[0]
    edges = []

    # Identify edges
    for i in range(nodes):
        for j in range(i + 1, nodes):
            if A[i, j] == 1:
                edges.append((i, j))
    
    # Initialize the constraint matrix
    U = np.zeros((nodes, len(edges)), dtype=int)
    
    # Fill in the constraint matrix
    for k, (i, j) in enumerate(edges):
        U[i, k] = 1
        U[j, k] = 1
    
    return U

# Example adjacency matrix
A = np.array([
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [0, 1, 0, 0]
])

U = adjacency_to_constraint_matrix(A)
print(U)

[[1 1 0 0]
 [1 0 1 1]
 [0 1 1 0]
 [0 0 0 1]]


In [ ]:
    def create_adjacency_matrix(self) -> np.array:
        # collect unique nodes
        nodes = set()
        for measurement in self.measurements:
            nodes.add(measurement.start)
            nodes.add(measurement.eind)
            
        nodes = sorted(list(nodes))
        
        node_indices = {node: idx for idx, node in enumerate(nodes)}
        
        num_nodes = len(nodes)
        adjacency_matrix = np.zeros((num_nodes,num_nodes), dtype=int)
        
        for measurement in self.measurements:
            start_idx = node_indices[measurement.start]
            eind_idx = node_indices[measurement.eind]
            adjacency_matrix[start_idx, eind_idx] = 1
        
        return adjacency_matrix
    
    def create_u_i(self,adjacency) -> np.array:
        nodes = adjacency.shape[0]
        edges = []

        # Identify edges
        for i in range(nodes):
            for j in range(i + 1, nodes):
                if adjacency[i, j] == 1:
                    edges.append((i, j))

        # Initialize the constraint matrix
        U = np.zeros((nodes, len(edges)), dtype=int)

        # Fill in the constraint matrix
        for k, (i, j) in enumerate(edges):
            U[i, k] = 1
            U[j, k] = 1

        return U

In [ ]:
import re

def find_largest_number(string):
    # Extract all numbers from the string using regular expression
    numbers = re.findall(r'\d+', string)
    
    # Convert the extracted numbers to integers
    int_numbers = [int(num) for num in numbers]
    
    # Find the largest number
    largest_number = max(int_numbers)
    
    return largest_number

max_val = 0
for i in solutions:
    largest_number = find_largest_number(i)
    if largest_number > max_val:
        max_val = largest_number

print(f"number of required qubits is {max_val+1}")

In [2]:
# number of qubits

import numpy as np

x_1 = 16
x_2 = 16.256
x_3 = 1

i_4 = np.array([0,2,0,1])
max_val = np.max(i_4) # get highest value from vector
# print(max_val)


num_digits = len(str(max_val))
for m in range(5):
    for i in range(2):
        num_x = 2**num_digits * i

solutions = []

def generate_pattern(start,multiples):
    pattern = ""
    val = [x for x in range(multiples)]
    qubit = start
    for index in val:
        if index == 0:
            positive_term = f"{2**(index)}q_{(qubit)}"
            qubit += 1
            negative_term = f"{2**(index)}q_{(qubit)}"
            qubit += 1
        else:
            positive_term = f"{2**(index)}q_{(qubit)}"
            qubit += 1
            negative_term = f"{2**(index)}q_{(qubit)}"
            qubit += 1
        
        if index > 0:
            pattern += " + " + positive_term + " - " + negative_term
        else:
            pattern += positive_term + " - " + negative_term
    return pattern

# Generate and print the pattern
start = 0
length_max_val_b = 2 * 2 * 4

val = [x for x in range(length_max_val_b)]
for i in val:
    if i == 0:
        pattern = generate_pattern(start,length_max_val_b)
        print("--------------------------------")
        print(pattern)
    elif i == 1:
        pattern = generate_pattern(length_max_val_b*2+1,length_max_val_b)
        print("--------------------------------")
        print(pattern)
    else:
        pattern = generate_pattern(length_max_val_b*2*i+1,length_max_val_b)
        print("--------------------------------")
        print(pattern)
        
# pattern2 = generate_pattern(length_max_val_b*2+1,length_max_val_b)
# print("--------------------------------")
# pattern3 = generate_pattern(length_max_val_b*4+1,length_max_val_b)
# print(pattern)
# print(pattern2)
# print(pattern3)


--------------------------------
1q_0 - 1q_1 + 2q_2 - 2q_3 + 4q_4 - 4q_5 + 8q_6 - 8q_7 + 16q_8 - 16q_9 + 32q_10 - 32q_11 + 64q_12 - 64q_13 + 128q_14 - 128q_15 + 256q_16 - 256q_17 + 512q_18 - 512q_19 + 1024q_20 - 1024q_21 + 2048q_22 - 2048q_23 + 4096q_24 - 4096q_25 + 8192q_26 - 8192q_27 + 16384q_28 - 16384q_29 + 32768q_30 - 32768q_31
--------------------------------
1q_33 - 1q_34 + 2q_35 - 2q_36 + 4q_37 - 4q_38 + 8q_39 - 8q_40 + 16q_41 - 16q_42 + 32q_43 - 32q_44 + 64q_45 - 64q_46 + 128q_47 - 128q_48 + 256q_49 - 256q_50 + 512q_51 - 512q_52 + 1024q_53 - 1024q_54 + 2048q_55 - 2048q_56 + 4096q_57 - 4096q_58 + 8192q_59 - 8192q_60 + 16384q_61 - 16384q_62 + 32768q_63 - 32768q_64
--------------------------------
1q_65 - 1q_66 + 2q_67 - 2q_68 + 4q_69 - 4q_70 + 8q_71 - 8q_72 + 16q_73 - 16q_74 + 32q_75 - 32q_76 + 64q_77 - 64q_78 + 128q_79 - 128q_80 + 256q_81 - 256q_82 + 512q_83 - 512q_84 + 1024q_85 - 1024q_86 + 2048q_87 - 2048q_88 + 4096q_89 - 4096q_90 + 8192q_91 - 8192q_92 + 16384q_93 - 16384q_94

In [8]:
30 / (5 * 2)


3.0

In [35]:
import re

def extract_q_terms(strings):
    q_dict = {}

    for i, string in enumerate(strings):
        key = f"x_{i+1}"
        terms = re.findall(r'([+-]?\d+)q(\d+)', string)
        tuples = []

        for term in terms:
            coefficient, q_index = term
            coefficient = int(coefficient)
            if int(q_index) % 2 == 1:
                tuples.append((coefficient,"-", f"q{q_index}"))
            else:
                tuples.append((coefficient,"+", f"q{q_index}"))

        q_dict[key] = tuples

    return q_dict

# List of strings
strings = ['1q0 - 1q1 + 2q2 - 2q3 + 4q4 - 4q5',
           '1q6 - 1q7 + 2q8 - 2q9 + 4q10 - 4q11',
           '1q12 - 1q13 + 2q14 - 2q15 + 4q16 - 4q17',
           '1q18 - 1q19 + 2q20 - 2q21 + 4q22 - 4q23',
           '1q24 - 1q25 + 2q26 - 2q27 + 4q28 - 4q29']

# Extract q_x terms into a single dictionary with tuples
q_dict = extract_q_terms(strings)

# Print the resulting dictionary
for key, value in q_dict.items():
    print(f"{key}: {value}")

x_1: [(1, '+', 'q0'), (1, '-', 'q1'), (2, '+', 'q2'), (2, '-', 'q3'), (4, '+', 'q4'), (4, '-', 'q5')]
x_2: [(1, '+', 'q6'), (1, '-', 'q7'), (2, '+', 'q8'), (2, '-', 'q9'), (4, '+', 'q10'), (4, '-', 'q11')]
x_3: [(1, '+', 'q12'), (1, '-', 'q13'), (2, '+', 'q14'), (2, '-', 'q15'), (4, '+', 'q16'), (4, '-', 'q17')]
x_4: [(1, '+', 'q18'), (1, '-', 'q19'), (2, '+', 'q20'), (2, '-', 'q21'), (4, '+', 'q22'), (4, '-', 'q23')]
x_5: [(1, '+', 'q24'), (1, '-', 'q25'), (2, '+', 'q26'), (2, '-', 'q27'), (4, '+', 'q28'), (4, '-', 'q29')]


In [40]:
import neal

# Define a simple QUBO
Q = {(0, 0): -5, (1, 1): -1, (0, 1): 2}

# Initialize the Neal sampler
sampler = neal.SimulatedAnnealingSampler()

# Sample from the QUBO
response = sampler.sample_qubo(Q, num_reads=5)

lowest_energy_sample = None
lowest_energy = float('-inf')

for sample, energy in response.data(['sample', 'energy']):
    if energy > lowest_energy:
        lowest_energy = energy
        lowest_energy_sample = sample
        
# Print the sample with the lowest energy
print("Sample with lowest energy:", lowest_energy_sample)
print("Energy:", lowest_energy)

Sample with lowest energy: {0: 1, 1: 0}
Energy: -5.0


In [41]:
arra = np.zeros([5,1])

for i in range(5):
    arra[i] = i
    
print(arra)

[[0.]
 [1.]
 [2.]
 [3.]
 [4.]]


In [57]:
def generate_pattern(start,multiples):
    pattern = ""
    val = [x for x in range(multiples)]
    qubit = start
    for i,index in enumerate(val):
        
        positive_term = f"{2**(index)}q_{(qubit)}"
        qubit += 1
        
        if i == 0:
            pattern += positive_term
        elif i % 2 == 1:
            pattern += " + " + positive_term
        elif i % 2 == 0:
            pattern += " - " + positive_term
    return pattern

# Generate and print the pattern
start = 0
length_max_val_b = 6

val = [x for x in range(length_max_val_b)]
for i in val:
    if i == 0:
        pattern = generate_pattern(start,length_max_val_b)
        print("--------------------------------")
        print(pattern)
    elif i == 1:
        pattern = generate_pattern(length_max_val_b,length_max_val_b)
        print("--------------------------------")
        print(pattern)
    else:
        pattern = generate_pattern(length_max_val_b*i,length_max_val_b)
        print("--------------------------------")
        print(pattern)
        
# pattern2 = generate_pattern(length_max_val_b*2+1,length_max_val_b)
# print("--------------------------------")
# pattern3 = generate_pattern(length_max_val_b*4+1,length_max_val_b)
# print(pattern)
# print(pattern2)
# print(pattern3)

--------------------------------
1q_0 + 2q_1 - 4q_2 + 8q_3 - 16q_4 + 32q_5
--------------------------------
1q_6 + 2q_7 - 4q_8 + 8q_9 - 16q_10 + 32q_11
--------------------------------
1q_12 + 2q_13 - 4q_14 + 8q_15 - 16q_16 + 32q_17
--------------------------------
1q_18 + 2q_19 - 4q_20 + 8q_21 - 16q_22 + 32q_23
--------------------------------
1q_24 + 2q_25 - 4q_26 + 8q_27 - 16q_28 + 32q_29
--------------------------------
1q_30 + 2q_31 - 4q_32 + 8q_33 - 16q_34 + 32q_35
